In [3]:
import numpy as np
import scipy.sparse as sp

from src.utils import train_model, write_submission
from src.recommender_model import RecommenderModel

# Top Popularity Recommender Model
This notebook provides an implementation of a TopPop recommender model.

## TopPop Recommender

In [2]:
class TopPopRecommender(RecommenderModel):
	def __init__(self):
		super(TopPopRecommender, self).__init__()
		self.urm_train: sp.csr_matrix | None = None
		self.items_popularity: np.ndarray | None = None

	def fit(self, urm: sp.csr_matrix, icm: sp.csr_matrix) -> None:
		self.urm = urm
		self.items_popularity = np.ediff1d(urm.tocsc().indptr)

	def _get_recommendations_predictions(self, user_id: int) -> np.ndarray:
		return self.items_popularity

In [3]:
top_pop_train = train_model(TopPopRecommender(), test_size=0.2)

In [ ]:
top_pop_submission = train_model(TopPopRecommender(), test_size=0)
write_submission(top_pop_submission, "top_pop_recommendation.csv")